# Shell Hackathon Solution 2019

This is the same code as for the solution for 2018, except that we use the predicted electricity demand of 2019 as input. We also create an output file with our solution for the infrastructure to be build, which we use then as input for 
the prediction for 2020.

In [1]:
from math import sqrt
from itertools import product

import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
supply = pd.read_csv("../data/raw/exisiting_EV_infrastructure_2018.csv")
demand = pd.read_csv("../data/processed/Demand_Future.csv")

In [3]:
slow_charger = 200
slow_costs = 1.0*600
fast_charger = 400
fast_costs = 1.5*600

In [4]:
# real capacities
real_capacities = (supply["existing_num_SCS"]*slow_charger + supply["existing_num_FCS"]*fast_charger).tolist()
# maximum theoretical capacities
capacities = (supply["total_parking_slots"]*fast_charger).tolist()
demands = demand["2019"].tolist()

In [5]:
sum(capacities) > sum(demands), sum(capacities), sum(demands), sum(real_capacities)

(True, 1000000, 444363.90464364795, 361600)

Also for 2019 the maximal capacity of all parking slots is more than enough to fulfill the demand. 

In [7]:
# facilities are parking slots
facilities = list(supply[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))
# customers are areas of electricity demand
customers = list(demand[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))

In [8]:
# Compute key parameters of MIP model formulation
num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [9]:
# This function determines the Euclidean distance between a facility and customer sites.
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [11]:
# Compute distance matrix
distance = {(c,f): compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [12]:
# maximum slots for chargers
slots = supply["total_parking_slots"].tolist()

# existing chargers
slow_slots = supply["existing_num_SCS"].tolist()
fast_slots = supply["existing_num_FCS"].tolist()

In [13]:
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [14]:
# Variables
assign = {}
for i,j in distance.keys(): 
    assign[(i,j)] = solver.NumVar(0,solver.infinity(),"Assign")
# Do not delete existing chargers (Constraint/Boundaries 2) 
slow = {}
for j in range(num_facilities):
    slow[j] = solver.IntVar(slow_slots[j], solver.infinity(), "Slow")
fast = {}
for j in range(num_facilities):
    fast[j] = solver.IntVar(fast_slots[j], solver.infinity(), "Fast")          

In [15]:
# Constraint 3 (slots)
for j in range(num_facilities):
    solver.Add(slow[j] + fast[j] <= slots[j])
# Constraint 5 (capacity constraints)
for j in range(num_facilities):
    solver.Add(sum(assign[(i,j)] for i in range(num_customers)) <= (slow[j]*200 + fast[j]*400))
# Constraint 6 (demand constraints)
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == demands[i] )    

In [16]:
objective = solver.Objective()
# Building costs for chargers
for j in range(num_facilities):
    objective.SetCoefficient(slow[j], slow_costs)
for j in range(num_facilities):
    objective.SetCoefficient(fast[j], fast_costs)
# distance costs
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],distance[(i,j)]) 
objective.SetMinimization()    

The Shell Hackathon cost function includes the MAE of the demand prediction as additional term. We know the exact demand for 2018, so this term is zero. For 2019/2020 we cannot compute the term, because we only have the predicted values, but not the exact ones.

In [17]:
status = solver.Solve()

In [18]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 2778785.7865769984

Problem solved in 245051.000000 milliseconds
Problem solved in 21549 iterations
Problem solved in 2 branch-and-bound nodes


In [30]:
# display optimal values of decision variables
lines = []
for j in range(num_facilities):
    build_slow = slow[j].solution_value() - slow_slots[j]
    build_fast = fast[j].solution_value() - fast_slots[j]
    if build_slow > 0 or build_fast > 0: 
        lines.append(f"Build {slow[j].solution_value() - slow_slots[j]} slow charger  and {fast[j].solution_value() - fast_slots[j]} fast charger at location {j + 1}.")
#for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)
    
print("Number of construction sites:", len(lines))          

...
Build 0.0 slow charger  and 10.0 fast charger at location 91.
Build 0.0 slow charger  and 10.0 fast charger at location 93.
Build 0.0 slow charger  and 12.0 fast charger at location 96.
Build 0.0 slow charger  and 14.0 fast charger at location 97.
Build 0.0 slow charger  and 14.0 fast charger at location 98.
Number of construction sites: 33


According to our solver we should build additional chargers at 33 parking slots.

In [20]:
# compute the total electricity supply of our solution
total_supply = 0
for j in range(num_facilities):
    total_supply += slow[j].solution_value()*200 + fast[j].solution_value()*400
sum(demands), total_supply     

(444363.90464364795, 473600.0)

In 2019 our total supply is closer to the predicted demand than in 2018.

In [21]:
# Shipments from facilities to customers.
lines = []
for i, j in assign.keys():
    if (abs(assign[i, j].solution_value()) > 1e-6):
        lines.append(f"Demand point {i + 1} receives {round(assign[i, j].solution_value(), 2)} of its demand {round(demands[i],2)} from parking slot {j + 1} .")
for line in lines[:5]: print(line)
print("...")
for line in lines[-5:]: print(line)        

Demand point 1 receives 19.36 of its demand 19.36 from parking slot 39 .
Demand point 2 receives 19.9 of its demand 19.9 from parking slot 39 .
Demand point 3 receives 19.39 of its demand 19.39 from parking slot 39 .
Demand point 4 receives 21.28 of its demand 21.28 from parking slot 39 .
Demand point 5 receives 20.79 of its demand 20.79 from parking slot 39 .
...
Demand point 4092 receives 8.41 of its demand 8.41 from parking slot 3 .
Demand point 4093 receives 2.58 of its demand 2.58 from parking slot 3 .
Demand point 4094 receives 4.79 of its demand 4.79 from parking slot 3 .
Demand point 4095 receives 9.39 of its demand 9.39 from parking slot 17 .
Demand point 4096 receives 10.35 of its demand 10.35 from parking slot 17 .


### Creating result file for submission

This file will later be used to create the final submission.

In [22]:
slow_list = []
for j in range(num_facilities):
    slow_list.append((2019, "SCS", "", j, slow[j].solution_value()))
fast_list = []
for j in range(num_facilities):
    fast_list.append((2019, "FCS", "", j, fast[j].solution_value()))    

In [23]:
assign_list = []
for i in range(num_customers):
    for j in range(num_facilities):
        assign_list.append((2019, "DS", i, j, assign[(i,j)].solution_value()))

In [24]:
result = slow_list + fast_list + assign_list

In [28]:
df_result = pd.DataFrame(result, columns=["year", "data_type", "demand_point_index", "supply_point_index", "value"])
df_result

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,,0,5.0
1,2019,SCS,,1,4.0
2,2019,SCS,,2,6.0
3,2019,SCS,,3,5.0
4,2019,SCS,,4,11.0
...,...,...,...,...,...
409795,2019,DS,4095,95,0.0
409796,2019,DS,4095,96,0.0
409797,2019,DS,4095,97,0.0
409798,2019,DS,4095,98,0.0


In [39]:
# commented out, so we don't overwrite the result
# df_result.to_csv("../data/processed/result_2019.csv")

### Creating infrastructure file for 2019 

We will use this file as basis for the optimization solution in 2020. This will make sure that we won't remove any chargers in 2020 compared to 2019.

In [27]:
supply_2019 = supply.copy()
supply_2019["existing_num_SCS"] = [entry[4] for entry in slow_list]
supply_2019["existing_num_FCS"] = [entry[4] for entry in fast_list]  
supply_2019

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
0,0,50.163110,19.412014,23,5.0,18.0
1,1,37.336451,58.119225,27,4.0,7.0
2,2,46.709232,57.525650,31,6.0,14.0
3,3,30.528626,55.379835,26,5.0,5.0
4,4,51.521781,35.116755,32,11.0,6.0
...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3.0,16.0
96,96,30.318396,33.388335,32,5.0,24.0
97,97,36.218839,22.235766,32,4.0,28.0
98,98,42.936915,38.122442,28,7.0,5.0


In [53]:
# commented out, so we don't overwrite the result
# supply_2019.to_csv("../data/processed/exisiting_EV_infrastructure_2019.csv")